<a href="https://colab.research.google.com/github/AkankshaTyagi11/hw_4/blob/main/Akanksha_Tyagi_HW_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import cv2
from torchvision.models.detection import fasterrcnn_mobilenet_v3_large_fpn, FasterRCNN_MobileNet_V3_Large_FPN_Weights
from torchvision.utils import draw_bounding_boxes
from torchvision.io.image import read_image
import matplotlib.pyplot as plt
from PIL import Image



from google.colab import drive

torch.manual_seed(42)

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


# Part 1

## Step 1. Collect a source video. It may be necessary to divide the video into discrete image frames.

Please find the collected video in the zip file. It's named as cars.mov. We directly load the video using opencv and run inference on each image frame.

# Initialze a pretrained Faster RCNN model with MobilenetV3 backbone.

In [ ]:
weights = FasterRCNN_MobileNet_V3_Large_FPN_Weights.COCO_V1
mobilenet_frcnn = fasterrcnn_mobilenet_v3_large_fpn(weights=weights, box_score_thresh=0.5)
mobilenet_frcnn.eval()
preprocess = weights.transforms()

## Step 2. Conduct inference on each frame of the video, drawing bounding boxes around detected vehicles.


In [ ]:
# Open the video.
cam = cv2.VideoCapture("/content/drive/MyDrive/cars.mov")

current_frame = 0

# Run inference on each frame.
images_with_boxes = []
while(True):

    # reading from frame
    ret, frame = cam.read()

    if ret:
      # Convert to RGB
      frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

      # Create model input
      img = torch.from_numpy(frame).permute(2, 0, 1)
      batch = [preprocess(img)]

      with torch.no_grad()
        # Run inference.
        prediction = mobilenet_frcnn(batch)[0]

      # Draw boxes on the frame
      labels = [weights.meta["categories"][i] for i in prediction["labels"]]
      img_box = draw_bounding_boxes(img, boxes=prediction["boxes"],
                              labels=labels,
                              colors="red",
                              width=1)

      # Save the frame with boxes in a list
      img_box_np = img_box.permute(1, 2, 0).cpu().numpy()
      images_with_boxes.append(img_box_np)

      current_frame += 1
      print(f'Running inference on frame no. {current_frame}')
    else:
        break

# Release all space and windows once done
cam.release()
cv2.destroyAllWindows()

Running inference on frame no. 1
Running inference on frame no. 2
Running inference on frame no. 3
Running inference on frame no. 4
Running inference on frame no. 5
Running inference on frame no. 6
Running inference on frame no. 7
Running inference on frame no. 8
Running inference on frame no. 9
Running inference on frame no. 10
Running inference on frame no. 11
Running inference on frame no. 12
Running inference on frame no. 13
Running inference on frame no. 14
Running inference on frame no. 15
Running inference on frame no. 16
Running inference on frame no. 17
Running inference on frame no. 18
Running inference on frame no. 19
Running inference on frame no. 20
Running inference on frame no. 21
Running inference on frame no. 22
Running inference on frame no. 23
Running inference on frame no. 24
Running inference on frame no. 25
Running inference on frame no. 26
Running inference on frame no. 27
Running inference on frame no. 28
Running inference on frame no. 29
Running inference on fr

  ## Step 3. Format the results back into a video.

In [ ]:
# Save the video
H, W = images_with_boxes[0].shape[:2]

out = cv2.VideoWriter('/content/drive/MyDrive/cars_boxes.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 30, (W, H))

for i in range(len(images_with_boxes)):
  img = cv2.cvtColor(images_with_boxes[i], cv2.COLOR_RGB2BGR)
  out.write(img)
out.release()
cv2.destroyAllWindows()